In [103]:
import math
import numpy as np

# look for an element in \mathbb{Z}\sqrt{q} such that
# >1, one hopes relation between conjugate and inverse as well
# myLambda2 is myLambda's conjugate
# could make this choice better for greater efficiency
# that's why put it into its own function
def findMyLambda(q):
    return [1+math.sqrt(q),1-math.sqrt(q)]

# for given number ring \mathbb{Z}[\sqrt{q}] shrink/expand the intervals by
#    factors of myLambda such that solutions in the new intervals are simply
#    the appropriate power of myLambda related to the solutions in the old intervals
#    In particular make the interval x0,x1 into length something between 1/myLambda and 1
def adjustIntervals(x0,x1,y0,y1,q):
    delta1=x1-x0
    delta2=y1-y0
    if (q==2):
        myLambda=1+math.sqrt(2)
        myLambda2=-1/myLambda
    else:
        [myLambda,myLambda2]=findMyLambda(q)
    #print(myLambda)
    # made such that inverse of myLambda is
    # negative of the conjugate, TODO: solve for general q
    newx0=x0
    newx1=x1
    newy0=y0
    newy1=y1
    # turn into the stadard form where the interval [x0,x1]
    #  has size somewhere between lambda^(-1) and 1
    expanded=True
    for i in range(10):
        if (delta1>=1):
            # shrink A and expand B
            newx0=newx0/myLambda
            newx1=newx1/myLambda
            newy0=newy0/myLambda2
            newy1=newy1/myLambda2
            delta1=delta1/myLambda
            delta2=delta2/myLambda2
            expanded=False
            #print("Shrinking"+str(delta1))
        elif (delta1<1/myLambda):
            # shrink B and expand A
            newx0=newx0*myLambda
            newx1=newx1*myLambda
            newy0=newy0*myLambda2
            newy1=newy1*myLambda2
            delta1=delta1*myLambda
            delta2=delta2*myLambda2
            #print("Expanding"+str(delta1))
        else:
            break
    newx=[newx0,newx1]
    newx.sort()
    [newx0,newx1]=newx
    newy=[newy0,newy1]
    newy.sort()
    [newy0,newy1]=newy
    if (expanded):
        i=-i
    # the solution for original problem will be 
    #    myLambda^i times the one found for the newx's and newy's
    return np.array([newx0,newx1,newy0,newy1,myLambda**i])

# assuming a<b and b-a<1, return the integer in this interval
def integerInRange(a,b):
    if (math.ceil(a)<=math.floor(b)):
        return math.ceil(a)
    else:
        return None

# https://arxiv.org/pdf/1403.2975.pdf proposition 4.5
# generalize to Z[\sqrt{q}] instead of Z[\sqrt{2}]
# with q a natural number q \geq 2
# returns a number alpha \in \mathbb{Z}[\sqrt{q}] between x0 and x1 such that
#    its galois conjugate is between y0 and y1
def gridProblem(x0,x1,y0,y1,q=2):
    newData=adjustIntervals(x0,x1,y0,y1,q)
    #print(newData)
    [newx0,newx1,newy0,newy1,adjustingFactor]=newData
    # if remove the print statement change to below
    #[newx0,newx1,newy0,newy1,adjustingFactor]=adjustIntervals(x0,x1,y0,y1,q)
    foundSolution=[]
    # foundSolution will be of the form a+b\sqrt{q}
    # with at most a single valid a for a given b
    # it will be the integer in [x0-b\sqrt{q},x1-b\sqrt{q}] if it exists
    # b will be between (x_0-y_1)/(2*sqrt{q}) and (x_1-y_0)/(2*sqrt{q})
    bRange1=(newx0-newy1)/(2*math.sqrt(q))
    bRange2=(newx1-newy0)/(2*math.sqrt(q))
    bottomRange=math.ceil(min(bRange1,bRange2))
    topRange=math.floor(max(bRange1,bRange2))
    for b in range(bottomRange,topRange+1):
        a = integerInRange(newx0-b*math.sqrt(q),newx1-b*math.sqrt(q))
        if (not (a is None)):
            #print("%i + %i \sqrt{%i}" % (a,b,q))
            newSol=a+b*math.sqrt(q)
            foundSolution.append(newSol)
    foundSolution.sort()
    return list(map(lambda x: adjustingFactor*x,foundSolution))

In [104]:
# example of looking for elements in Z[\sqrt{5}] between -2 and 5
# with conjugates between -100 and 100
gridProblem(-2,5,-100,100,5)

[-1.8885438199983233,
 -1.7507764050037717,
 -1.3049516849970659,
 -1.1671842700025146,
 -0.72135954999580876,
 -0.58359213500125728,
 0.0,
 0.58359213500125728,
 0.72135954999580876,
 1.1671842700025146,
 1.3049516849970659,
 1.7507764050037717,
 1.8885438199983233,
 2.4721359549995805,
 3.0557280900008377,
 3.1934955049953895,
 3.6393202250020953,
 3.7770876399966467,
 4.2229123600033525,
 4.3606797749979043,
 4.944271909999161]